<img align="left" src = "images/linea.png" width=100 style="padding: 30px"> 
<img align="left" src = "images/rubin.png" width=180 style="padding: 20px"> 

# Photo-z Server - Tutorial Notebook

Contact author: [Julia Gschwend](mailto:julia@linea.org.br)

Last verified run: 2023-02-27 <br>

<a class="anchor" id="contents"></a>
#### Notebook contents
- PZ Server
    - [Introduction](#intro) 
    - [How to upload a data product to the PZ Server](#upload)
- PZ Server API (Python library pz-server-lib)
    - [Get basic info from PZ Server](#basic-info)
    - [How to display the metadata of a data product](#metadata)
    - [How to download data products as .zip files](#download-zip)  
    - [How to retrieve contents of data products (work on memory)](#retrieve-contents)
        - [Spec-z Catalogs](#spec)
        - [Training Sets](#train)
        - [Photo-z Validation Results](#valid)
        - [Photo-z Tables](#pz_tables)

<a class="anchor" id="intro"></a>
# PZ Server
## Introduction 


The Photo-z (PZ) Server is an online service available for the LSST Community to host and share lightweight photo-z related data products. The upload and download of data and metadata can be done at the website [pz-server.linea.org.br](https://pz-server.linea.org.br/) (during the development phase, a test environment is available at [pz-server-dev.linea.org.br](https://pz-server-dev.linea.org.br/). There, you will find two separate pages containing a list of data products each: one for LSST Data Management's oficial data products, and other for user-generated data products. **The registered data products can also be accessed directly from Python code using the PZ Server's data access API, as demonstrated below.**

The PZ Server is developed and delivered as part of the in-kind contribution program BRA-LIN, from LIneA to the Rubin Observatory's LSST. The service is hosted in the Brazilian IDAC, not directly connected to the [Rubin Science Platform (RSP)](https://data.lsst.cloud/). However, it requires RSP credentials for user's authentication. For a comprehensive documentation about the PZ Server, please visit the [PZ Server's documentation page](https://linea-it.github.io/pz-lsst-inkind-doc/). There, you will find also an overview of all LIneA's contributions related to Photo-zs. The internal documentation of the API functions is available on the [API's documentation page](https://linea-it.github.io/pz-server-lib/html/index.html).  

<a class="anchor" id="upload"></a>
### How to upload a data product to the PZ Server
<div style="text-align: right"><a href="#contents">back to the top</a></div> 

To upload a data product, click on the button **NEW PRODUCT** on the top left of the **User-generated Data Products** page and fill in the Upload Form with relevant metadata.

The photo-z-related products are organized into five categories (product types):

- **Spec-z Catalog:** Catalog of spectroscopic redshifts and positions (usually equatorial coordinates).
- **Training Set:** Training set for photo-z algorithms (tabular data). It usually contains magnitudes, errors, and true redshifts.
- **Photo-z Validation Results:** Results of a photo-z validation procedure (free format). Usually contains photo-z estimates (single estimates and/or pdf) of a validation set, photo-z validation metrics, validation plots, etc.
- **Photo-z Table:** Results of a photo-z estimation procedure. Ideally in the same format as the photo-z tables delivered by the DM as part of the LSST data releases. If the data is larger than the file upload limit (200MB), the product entry stores only the metadata (and instructions on accessing the data should be provided in the description field).

# The PZ Server API (Python library pz-server-lib)
<div style="text-align: right"><a href="#contents">back to the top</a></div> 

### Installation

**Using pip**

The PZ Server API is avalialble on **pip** as  `pz-server-lib`. To install the API and its dependencies, type, on the Terminal:  

```
$ pip install pz-server-lib 
``` 

**Using setup.py**  

Alternatively, if you have cloned the repository with:

```
$ git clone https://github.com/linea-it/pz-server-lib.git  
``` 

To install the API and its dependencies, type:

```
$ python setup.py install
```


OBS: You might need to restart the kernel on the notebook to incorporate the new library. 


### Imports and Setup

In [ ]:
from pz_server import PzServer 
%reload_ext autoreload 
%autoreload 2

The connection with the PZ Server from Python code is done by an object of the class `PzServer`. To get authorization to define an instance of `PzServer`, the users must provide an **API Token** generated on the top right menu on the [PZ Server website](https://pz-server.linea.org.br/). 

<img src="../images/ScreenShotTokenMenu.png" width=150pt align="top"/> <img src="../images/ScreenShotTokenGenerator.png" width=300pt/>

In [ ]:
pz_server = PzServer(token="<paste your API Token here>", host="pz-dev") # temporary host for test phase  

In [ ]:
pz_server = PzServer(token="1a08b1340c201c8f58c90e9fa9260effbd649439", host="pz-dev") # temporary host for test phase  

<a class="anchor" id="basic-info"></a>
## Get basic info from PZ Server
<div style="text-align: right"> <a href="#contents">back to the top</a> </div> 

The Python object `pz_server` just created above can provide access to data and metadata stored in the PZ Server. It also brings useful methods for users to navigate through the available contents. The methods with the preffix "get_" return the result of a query on the PZ Server database as a Python dictionary, and are most useful to be used programatically (see detaials on the [API documentation page](https://linea-it.github.io/pz-server-lib/html/index.html)). Those with the preffix "display_" just show the results as a styled [_Pandas DataFrames_](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (optimized for Jupyter Notebook). For instance:

Display the list of product types supported with a short description;

In [ ]:
pz_server.display_product_types()

Display the list of users who uploaded data products to the server;

In [ ]:
pz_server.display_users()

Display the list of data releases available at the time; 

In [ ]:
pz_server.display_releases()

---
### Data products 
Display all data products available (WARNING: this list can rapdly grow during the survey's operation). 

In [ ]:
pz_server.display_products_list() 

The information about product type, users, and releases shown above can be used to filter the data products of interest for your search. For that, the method `list_products` receives as argument a dictionary mapping the products attributes to their values. 

In [ ]:
pz_server.display_products_list(filters={"release": "LSST DP0", 
                                 "product_type": "Training Set",
                                 "uploaded_by": "Gschwend"})

It also works if we type a string pattern that is part of the value. For instance, just "DP0" instead of "LSST DP0": 

In [ ]:
pz_server.display_products_list(filters={"release": "DP0"})

It also allows the search for multiple strings by adding the suffix `__or` (two underscores + "or") to the search key. For instance, to get spec-z catalogs and training sets in the same search (notice that filtering is not case sensitive):

In [ ]:
pz_server.display_products_list(filters={"product_type__or": ["Spec-z Catalog", "training set"]})

To fetch the results of a search and attribute to a variable, just change the preffix "display_" by "get_", like this:  

In [ ]:
search_results = pz_server.get_products_list(filters={"product_type": "results"}) # PZ Validation results
search_results

<a class="anchor" id="metadata"></a>
## How to display the metadata of a data product
<div style="text-align: right">  <a href="#contents">back to the top</a> </div>     

The metadata of a given data product is the information provided by the user on the upload form. This information is attached to the data product contents and is available for consulting on the PZ Server page or using this Python API (`pz-server-lib`).

All data products stored on PZ Server are identified by a unique name, a _string_ called **internal_name**, which is created automatically at the moment of the upload by concatenating the product name given by its owner (and removing blank spaces, upper cases, and special charecters) to a randomly generated hash. 

One way to share a data product with othes PZ Server users is to provide the product's URL:

https://pz-server.linea.org.br/product/<internal_name> 

or, if still in the development phase, 

https://pz-server-dev.linea.org.br/product/<internal_name> 

The `PzServer`'s method `get_product_metadata()` returns a dictionary with the attibutes stored in the PZ Server about a given data product identified by its **internal_name**. For use in a Jupyter notebook, the equivalent `display_product_metadata()` shows the results in a formated table.

In [ ]:
# pz_server.display_product_metadata(<internal name (str)>) 
pz_server.display_product_metadata("simple_training_set_Uch3wIA") 

As in the previous examples, to attribute the metadata of a data product (as a dictionary) to a variable, just replace the preffix "display_" by "get_", like this:  

In [ ]:
metadata_dic = pz_server.get_product_metadata("simple_training_set_Uch3wIA") 
metadata_dic

<a class="anchor" id="download-zip"></a>
## How to download data products as .zip files
<div style="text-align: right">  <a href="#contents">back to the top</a> </div>     

To download any data product stored in the PZ Server, use the `PzServer`'s method `download_product` informing the product's **internal_name** and the path to where it will be saved (the default is the current directory). This method downloads a compressed .zip file which contais all the files uploaded by the user, including data, anciliary files and description files. Let's try it with a small data product. 

In [ ]:
pz_server.download_product("gama_specz_subsample_sE", save_in=".")

<a class="anchor" id="retrieve-contents"></a>
## How to retrieve contents of data products (work on memory)
<div style="text-align: right"> <a href="#contents">back to the top</a> </div>     

Another feature of the PZ Server API is to retrieve the contents of a given data product and atribute it to a variable in Python code. This feature is available only for tabular data (product types: **Spec-z Catalog** and **Training Set**). Let's see an example with the same data product as shown above.      

In [ ]:
data = pz_server.get_product("gama_specz_subsample_sE")

<div style="text-align: right"> <a href="#contents">back to the top</a> </div> 
<a class="anchor" id="prod-types"></a>

## Product types

The PZ Server API provides Python classes with useful methods to handle particular product types.  

In [ ]:
pz_server.display_product_types()

<a class="anchor" id="spec"></a>
### Spec-z Catalog 
<div style="text-align: right">  <a href="#contents">back to the top</a> </div> 

In the context of the PZ Server, Spec-z Catalogs are defined as any catalog containing spherical equatorial coordinates and spectroscopic redshift measurements (or, analogously, true redshifts from simulations). A Spec-z Catalog can include data from a single spectroscopic survey or a combination of data from several sources and should be provided as a single file with tabular data to PZ Server's the upload tool. 

Mandatory columns: 
* Right ascension [degrees] - `float`
* Declination [degrees] - `float`
* Spectroscopic or true redshift - `float`

Recommended columns: 
* Spectroscopic redshift error - `float`
* Quality flag - `integer`, `float`, or `string`
* Survey name (recommended for compilations of data from different surveys)




#### List Spec-z Catalogs available on PZ Server

In [ ]:
pz_server.display_products_list(filters={"product_type": "Spec-z Catalog"})

#### Display metadata of a given Spec-z Catalog 

#### Retrieve a given Spec-z Catalog - UNDER CONSTRUCTION

The method `get_product()` returns the data stored under a given product **id** number or **internal_name** as an Astropy Table object (valid only for tabular data (e.g.: product types Spec-z Catalog and Training Set).   

In [ ]:
#specz_catalog = pz_server.get_product(<product id number or name>)
#specz_catalog = pz_server.get_product(6)
#specz_catalog # .to_pandas()

For simple tabular data (product types: **Spec-z Catalog** and **Training Set**), the method returns an object of a class corresponding to the product type (classes: SpeczCatalog or TrainingSet), which are simple extensions of the class _pandas.DataFrame_ with some particular attributes and methods). Let's see an axample with a Spec-z Catalog:

import pandas as pd
pd.__version__

In [ ]:
#specz_catalog.product_id

Display basic statistics

In [ ]:
#specz_catalog.describe()

Basic plots for quick visualization of catalog properties 

In [ ]:
#specz_catalog.plot()

In [ ]:
#specz_catalog.product_id

In [ ]:
#specz_catalog.attrs["prod_type"]

<a class="anchor" id="train"></a>
### Training Sets 
<div style="text-align: right">  <a href="#contents">back to the top</a> </div> 
    
In the context of the PZ Server, Training Sets are defined as the product of matching (spatially) a given Spec-z Catalog (single survey or compilation) to the photometric data, in this case, the LSST Objects Catalog. The PZ Server API offers a tool called _Training Set Maker_ for users to build customized Training Sets based on the Spec-z Catalogs available. Please see the companion Jupyter Notebook `pz_tsm_tutorial.ipynb` for details.   


_Note 1: Commonly the training set is split into two or more subsets for photo-z validation purposes. If the Training Set owner has previously defined which objects should belong to each subset (trainining and validation/test sets), this information must be available as an extra column in the table or as clear instructions for reproducing the subsets separation in the data product description._

  
_Note 2: The PZ Server only supports catalog-level Training Sets. Image-based Training Sets, e.g., for deep-learning algorithms, are not supported yet._


Mandatory column: 
* Spectroscopic (or true) redshift - `float`

Other expected columns
* Object ID from LSST Objects Catalog - `integer`
* Observables: magnitudes (and/or colors, or fluxes) from LSST Objects Catalog - `float`
* Observable errors: magnitude errors (and/or color errors, or flux errors) from LSST Objects Catalog - `float`
* Right ascension [degrees] - `float`
* Declination [degrees] - `float`
* Quality Flag - `integer`, `float`, or `string`
* Subset Flag - `integer`, `float`, or `string`



#### List Training Sets available on PZ Server

In [ ]:
#pz_server.display_products_list(filters={"product_type": "Training Set"})

#### Display metadata of a given Training Set

In [ ]:
#metadata_train_set = pz_server.get_product_metadata(9)
#pz_server.display_product_metadata("14_goldenspike_train") 

#### Retrieve a given Training Set 

In [ ]:
#train_set = pz_server.get_product(9)
#train_set

Display basic statistics

In [ ]:
#train_set.describe()

Use the methods `train_set_plots` for a quick visualization of training sets properties: 

In [ ]:
#train_set.plot(mag_name="mag_i", redshift_name="z_true")

<a class="anchor" id="valid"></a>
### Photo-z Validation Results
<div style="text-align: right">  <a href="#contents">back to the top</a> </div> 
    
Validation Results are the outputs of any photo-z algorithm applied on a Validation Set. The format and number of files of this data product are strongly dependent on the algorithm used to create it, so there are no constraints on these two parameters. In the case of multiple files, for instance, if the user includes the results of training procedures (e.g., neural nets weights, decision trees files, or any machine learning by-product) or additional files (SED templates, filter transmission curves, theoretical magnitudes grid, Bayesian priors, etc.), it will be required to put all files together in a single compressed file (.zip or .tar, or .tar.gz) before uploading it to the Photo-z Server.   

#### List Validation Results available on PZ Server

In [ ]:
#pz_server.display_products_list(filters={"product_type": "Validation Results"})

#### Display metadata of a given data product of Photo-z Validation Results

In [ ]:
#pz_server.display_product_metadata("21_pz_validation_goldenspike_fzboost")

#### Retrieve a given Photo-z Validation Results: download file

This product type is not necessarily (only) tabular data and can be a list of files. The methods `get_product` shown above just return the data to be used on memory and only supports single tabular files. To retrieve Photo-z Validation Results, you must download the data to open locally. 

In [ ]:
#pz_server.get_product(21)#, save_in=".")

In [ ]:
#pz_server.download_product(21, save_in=".")

<a class="anchor" id="pz_tables"></a>
### Photo-z Tables 
<div style="text-align: right">  <a href="#contents">back to the top</a> </div> 

The Photo-z Tables are the results of photo-z estimation on photometrics samples. The size limit for uploading files on the PZ Server is 200MB, therefore it does not support large Photo-z Tables such as the photo-zs of the LSST Objects catalog. Instead, the PZ Server can host only Photo-z Tables of small data sets. 

In [ ]:
#pz_server.download_product(<id number or internal name>)